In [1]:
import test_visualization as tv

PDB File:  TR217.pdb
PDB File:  TR228.pdb
PDB File:  TR274.pdb
PDB File:  TR280.pdb
PDB File:  TR283.pdb
PDB File:  TR759.pdb
PDB File:  TR760.pdb
PDB File:  TR762.pdb
PDB File:  TR765.pdb
PDB File:  TR768.pdb
PDB File:  TR769.pdb
PDB File:  TR772.pdb
PDB File:  TR774.pdb
PDB File:  TR776.pdb
PDB File:  TR780.pdb
PDB File:  TR782.pdb
PDB File:  TR783.pdb
PDB File:  TR786.pdb
PDB File:  TR792.pdb
PDB File:  TR795.pdb
PDB File:  TR803.pdb
PDB File:  TR810.pdb
PDB File:  TR811.pdb
PDB File:  TR816.pdb
PDB File:  TR817.pdb
PDB File:  TR821.pdb
PDB File:  TR822.pdb
PDB File:  TR823.pdb
PDB File:  TR827.pdb
PDB File:  TR828.pdb
PDB File:  TR829.pdb
PDB File:  TR833.pdb
PDB File:  TR837.pdb
PDB File:  TR848.pdb
PDB File:  TR854.pdb
PDB File:  TR856.pdb
PDB File:  TR857.pdb


In [3]:
tv.plot_contact_maps(tv.model, tv.fasta_seqs, tv.c_maps)